In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# 讀取CSV文件
df = pd.read_csv('/content/drive/MyDrive/archive/dogs.csv')

# 選擇指定類別
selected_breeds = [
    "Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua",
    "Collie", "Dingo", "French Bulldog", "German Sheperd",
    "Malinois", "Newfoundland", "Pekinese", "Pomeranian", "Pug", "Vizsla"
]

# 過濾指定的品種
df_selected = df[df['labels'].isin(selected_breeds)]

# 加載和預處理圖像
def load_images(df):
    image_list = []
    for idx, row in df.iterrows():
        img = Image.open('/content/drive/MyDrive/archive/' + row['filepaths'])
        img = img.resize((224, 224))  # VGG16預設的圖像大小為224x224
        img_array = np.array(img) / 255.0  # 圖像歸一化
        image_list.append(img_array)
    return np.array(image_list)

images = load_images(df_selected)

# 標籤編碼
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_selected['labels'])
labels = to_categorical(labels)  # 轉換為one-hot編碼

# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 加载预训练的VGG16模型
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 冻结卷积基

# 建立新模型
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(selected_breeds), activation='softmax')
])

# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# 訓練模型，使用数据增强
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=10,
          validation_data=(X_test, y_test))

# 評估模型
val_loss, val_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {val_acc}")
# accuracy = 0.665

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
51/51 [==============================] - 1276s 25s/step - loss: 3.8390 - accuracy: 0.1726 - val_loss: 1.9913 - val_accuracy: 0.3970
Epoch 2/10
51/51 [==============================] - 1280s 25s/step - loss: 2.2156 - accuracy: 0.2700 - val_loss: 1.7591 - val_accuracy: 0.5211
Epoch 3/10
51/51 [==============================] - 1267s 25s/step - loss: 1.9933 - accuracy: 0.3203 - val_loss: 1.5712 - val_accuracy: 0.5682
Epoch 4/10
51/51 [==============================] - 1264s 25s/step - loss: 1.8530 - accuracy: 0.3594 - val_loss: 1.3585 - val_accuracy: 0.6303
Epoch 5/10
51/51 [==============================] - 1238s 24s/step - loss: 1.7713 - accuracy: 0.3904 - val_loss: 1.2709 - val_accuracy: 0.6228
Epoch 6/10
51/51 [==============================] - 1237s 24s/step - loss: 1.7153 - accuracy: 0.3880 - val_loss: 1.1640 - val_accuracy: 0.6774
Epoch 7/10
51/51 [==============================] - 1264s 25s/step - loss: 1.